In [1]:
import os
import json
import sqlite3

In [5]:
def create_database(db_path):
    """
    Creates an SQLite database and table if they do not exist.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS music_data (
        artist_familiarity REAL,
        artist_hotttnesss REAL,
        artist_id TEXT,
        artist_latitude REAL,
        artist_location TEXT,
        artist_longitude REAL,
        artist_name TEXT,
        genre TEXT,
        release TEXT,
        song_hotttnesss REAL,
        song_id TEXT PRIMARY KEY,
        title TEXT,
        danceability REAL,
        duration REAL,
        end_of_fade_in REAL,
        energy REAL,
        song_key INTEGER,
        loudness REAL,
        mode INTEGER,
        start_of_fade_out REAL,
        tempo REAL,
        time_signature INTEGER,
        track_id TEXT,
        year INTEGER,
        artist_terms TEXT, -- JSON list
        similar_artists TEXT, -- JSON list
        bars_start TEXT, -- JSON list
        beats_start TEXT, -- JSON list
        sections_start TEXT, -- JSON list
        segments_start TEXT, -- JSON list
        tatums_start TEXT -- JSON list
    );
    """)
    
    conn.commit()
    conn.close()


In [4]:
# Example usage
database_path = "music_data.db"
json_root_directory = "/Users/johndriscoll/Downloads/lastfm_subset"  # The top-level directory where the recursive structure starts

create_database(database_path)  # Create database if not exists
insert_json_to_db(json_root_directory, database_path)  # Insert JSON files recursively


In [77]:
import sqlite3
import pandas as pd
import json
import h5py
import os

In [78]:
# SQLite database file
DB_FILE = "music_data.db"

In [79]:
# drop music_data table to clear data
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()
    
cursor.execute("""DROP TABLE music_data""")
conn.commit()
conn.close()

OperationalError: no such table: music_data

In [80]:
# Function to create the SQLite database & table
def create_database():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS music_data (
        artist_familiarity REAL,
        artist_hotttnesss REAL,
        artist_id TEXT,
        artist_latitude REAL,
        artist_location TEXT,
        artist_longitude REAL,
        artist_name TEXT,
        genre TEXT,
        release TEXT,
        song_hotttnesss REAL,
        song_id TEXT PRIMARY KEY,
        title TEXT,
        danceability REAL,
        duration REAL,
        end_of_fade_in REAL,
        energy REAL,
        song_key INTEGER,
        loudness REAL,
        mode INTEGER,
        start_of_fade_out REAL,
        tempo REAL,
        time_signature INTEGER,
        track_id TEXT,
        year INTEGER,
        artist_terms TEXT, -- JSON list
        similar_artists TEXT, -- JSON list
        bars_start TEXT, -- JSON list
        beats_start TEXT, -- JSON list
        sections_start TEXT, -- JSON list
        segments_start TEXT, -- JSON list
        tatums_start TEXT -- JSON list
    );
    """)
    
    conn.commit()
    conn.close()


In [95]:
def read_hdf5(file_path):
    with h5py.File(file_path, 'r') as h5:
        # Extract scalar values directly
        metadata = h5["metadata/songs"][0]  # First row
        #print(dict(zip(metadata.dtype.names, metadata)))
        analysis = h5["analysis/songs"][0]  # First row
        print(dict(zip(metadata.dtype.names, analysis)))
        year = h5["musicbrainz/songs"][0]  # First row

        # Convert list-based fields to JSON strings
        artist_terms = json.dumps([item.decode() for item in h5["metadata/artist_terms"][:]])
        similar_artists = json.dumps([item.decode() for item in h5["metadata/similar_artists"][:]])
        bars_start = json.dumps(h5["analysis/bars_start"][:].tolist())
        beats_start = json.dumps(h5["analysis/beats_start"][:].tolist())
        sections_start = json.dumps(h5["analysis/sections_start"][:].tolist())
        segments_start = json.dumps(h5["analysis/segments_start"][:].tolist())
        tatums_start = json.dumps(h5["analysis/tatums_start"][:].tolist())

        # do empty string handling on all .decode arguments
        artist_id = metadata["artist_id"].decode().strip()
        # Combine data into a dictionary (single row)
        song_data = {
            "artist_familiarity": float(metadata["artist_familiarity"]) if metadata["artist_familiarity"] else None,
            "artist_hotttnesss": float(metadata["artist_hotttnesss"]) if metadata["artist_hotttnesss"] else None,
            "artist_id": metadata["artist_id"].decode().strip() or None,
            "artist_latitude": float(metadata["artist_latitude"]) if metadata["artist_latitude"] else None,
            "artist_location": metadata["artist_location"].decode().strip() or None,
            "artist_longitude": float(metadata["artist_longitude"]) if metadata["artist_longitude"] else None,
            "artist_name": metadata["artist_name"].decode(),
            "genre": metadata["genre"].decode().strip() or None,
            "release": metadata["release"].decode().strip() or None,
            "song_hotttnesss": float(metadata["song_hotttnesss"]) if metadata["song_hotttnesss"] else None,
            "song_id": metadata["song_id"].decode().strip() or None,
            "title": metadata["title"].decode().strip() or None,
            "danceability": float(analysis["danceability"]) if "danceability" in analysis else None,
            "duration": float(analysis["duration"]) if "duration" in analysis else None,
            "end_of_fade_in": float(analysis["end_of_fade_in"]) if "end_of_fade_in" in analysis else None,
            "energy": float(analysis["energy"]) if "energy" in analysis else None,
            "song_key": int(analysis["key"]) if "key" in analysis else None,
            "loudness": float(analysis["loudness"]) if "loudness" in analysis else None,
            "mode": int(analysis["mode"]) if "mode" in analysis else None,
            "start_of_fade_out": float(analysis["start_of_fade_out"]) if "start_of_fade_out" in analysis else None,
            "tempo": float(analysis["tempo"]) if "tempo" in analysis else None,
            "time_signature": int(analysis["time_signature"]) if "time_signature" in analysis else None,
            "track_id": metadata["track_7digitalid"],
            "year": int(year["year"]) if "year" in year else None,
            "artist_terms": artist_terms,
            "similar_artists": similar_artists,
            "bars_start": bars_start,
            "beats_start": beats_start,
            "sections_start": sections_start,
            "segments_start": segments_start,
            "tatums_start": tatums_start
        }

        return song_data


In [98]:
# Function to process all HDF5 files in a folder
def process_hdf5_files(folder_path):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.h5'):
                song_data = read_hdf5(os.path.join(root, file))

                placeholders = ", ".join(["?"] * len(song_data))
                columns = ", ".join(song_data.keys())

                sql = f"INSERT OR IGNORE INTO music_data ({columns}) VALUES ({placeholders})"
                cursor.execute(sql, list(song_data.values()))
                # conn.commit()    
                # conn.close()
                # return
    conn.commit()    
    conn.close()


In [99]:
# ------------------------ RUN ETL PROCESS ------------------------ 
create_database()
process_hdf5_files("/Users/johndriscoll/Downloads/MillionSongSubset") 

#print("Data import complete!")

{'analyzer_version': 22050, 'artist_7digitalid': b'd8bafd4a65d1855aec08991c8b013dc1', 'artist_familiarity': 0.0, 'artist_hotttnesss': 148.74077, 'artist_id': 0.192, 'artist_latitude': 0.0, 'artist_location': 0, 'artist_longitude': 0, 'artist_mbid': 0, 'artist_name': 0, 'artist_playmeid': 0, 'genre': 0, 'idx_artist_terms': 0, 'idx_similar_artists': 0, 'release': 0, 'release_7digitalid': 0, 'song_hotttnesss': 0, 'song_id': 0, 'title': 0, 'track_7digitalid': 0}
{'analyzer_version': 22050, 'artist_7digitalid': b'55f60c97280172e9276723c06e531996', 'artist_familiarity': 0.0, 'artist_hotttnesss': 252.99546, 'artist_id': 0.514, 'artist_latitude': 0.0, 'artist_location': 0, 'artist_longitude': 0, 'artist_mbid': 0, 'artist_name': 0, 'artist_playmeid': 0, 'genre': 0, 'idx_artist_terms': 0, 'idx_similar_artists': 0, 'release': 0, 'release_7digitalid': 0, 'song_hotttnesss': 0, 'song_id': 0, 'title': 0, 'track_7digitalid': 0}
{'analyzer_version': 22050, 'artist_7digitalid': b'053fb50807248bef996e6c7

KeyboardInterrupt: 

In [84]:
conn = sqlite3.connect(DB_FILE)
#cursor = conn.cursor()
sql = "SELECT * FROM music_data LIMIT 5"
res = conn.execute(sql)
data = res.fetchall()
dict(zip([description[0] for description in res.description],data[0]))

{'artist_familiarity': 0.5574602197393447,
 'artist_hotttnesss': 0.3861516314132549,
 'artist_id': 'AREJXK41187B9A4ACC',
 'artist_latitude': 46.71067,
 'artist_location': 'France',
 'artist_longitude': 1.71819,
 'artist_name': 'Raphaël',
 'genre': None,
 'release': 'Je Sais Que La Terre Est Plate (Deluxe)',
 'song_hotttnesss': 0.5479529419800353,
 'song_id': 'SOGSMXL12A81C23D88',
 'title': 'Je Sais Que La Terre Est Plate',
 'danceability': None,
 'duration': None,
 'end_of_fade_in': None,
 'energy': None,
 'song_key': None,
 'loudness': None,
 'mode': None,
 'start_of_fade_out': None,
 'tempo': None,
 'time_signature': None,
 'track_id': b' \xe6.\x00',
 'year': None,
 'artist_terms': '["chanson", "visual kei", "hip hop", "pop rock", "british pop", "jrock", "world music", "downtempo", "singer-songwriter", "latin pop", "french", "male vocalist", "alternative rock", "chill-out", "rock", "french pop", "ethnic fusion", "electronica", "pop", "acoustic", "chanson francaise", "japanese", "j po

In [92]:
conn = sqlite3.connect(DB_FILE)
#cursor = conn.cursor()
sql = "SELECT COUNT(*) FROM music_data WHERE energy IS NULL"
res = conn.execute(sql)
data = res.fetchall()
data

[(10000,)]

In [85]:
pd.DataFrame(data=data, columns=[description[0] for description in res.description])

,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,genre,release,song_hotttnesss,...,time_signature,track_id,year,artist_terms,similar_artists,bars_start,beats_start,sections_start,segments_start,tatums_start
0,0.557460,0.386152,AREJXK41187B9A4ACC,46.71067,France,1.71819,Raphaël,None,Je Sais Que La Terre Est Plate (Deluxe),0.547953,...,None,b' \xe6.\x00',None,"[""chanson"", ""visual kei"", ""hip hop"", ""pop rock...","[""ARVEJ9M1187FB4DC44"", ""ARYDHN21187FB466A8"", ""...","[0.52458, 2.4444, 4.3658, 6.30006, 8.22086, 10...","[0.04257, 0.52458, 1.00659, 1.48632, 1.96605, ...","[0.0, 7.74174, 40.30707, 80.4797, 89.656, 127....","[0.0, 0.19188, 0.60499, 0.84939, 1.10018, 1.36...","[0.04257, 0.28357, 0.52458, 0.76558, 1.00659, ..."
1,0.626958,0.434860,AR2XRFQ1187FB417FE,NaN,None,NaN,Julie Zenatti,None,Comme Vous,0.475638,...,None,b'\x9c\xd6@\x00',None,"[""chanson"", ""dance pop"", ""pop rock"", ""soft roc...","[""ARAGSU61187B98C95C"", ""ARO9XCZ1187B99C255"", ""...","[8.69527, 12.01758, 14.81363, 20.09003, 22.693...","[0.59161, 1.33555, 2.07949, 2.82093, 3.56238, ...","[0.0, 7.08047, 39.03442, 89.26989, 114.94898, ...","[0.0, 0.51379, 0.89138, 1.30912, 1.65297, 2.05...","[0.59161, 0.84042, 1.08799, 1.33555, 1.58436, ..."
2,0.425724,NaN,ARODOO01187FB44F4A,NaN,None,NaN,The Baltimore Consort,None,Watkins Ale - Music of the English Renaissance,NaN,...,None,b'\x9f\xf2R\x00',None,"[""early music"", ""celtic"", ""mediaeval"", ""folk"",...","[""ARVZRHW11F4C83CF5F"", ""ARGCYMZ1187B9A6EA1"", ""...","[1.95344, 6.35701, 10.70579, 15.07913, 19.4832...","[1.95344, 3.0622, 4.16548, 5.26467, 6.35701, 7...","[0.0, 13.98322, 29.36585, 42.60077]","[0.0, 0.97365, 1.34431, 1.81351, 2.19764, 2.44...","[1.11537, 1.39701, 1.67864, 1.95344, 2.23097, ..."
3,0.611495,0.334520,ARJGW911187FB586CA,NaN,None,NaN,I Hate Sally,None,Don't Worry Lady,NaN,...,None,b'8O\x12\x00',None,"[""post-hardcore"", ""doomcore"", ""metalcore"", ""sc...","[""ARMJF6H1187B9A1798"", ""ARYOWT31187B9AB7AC"", ""...","[0.39309, 2.75109, 5.13649, 7.52978, 9.95582, ...","[0.39309, 1.17222, 1.95902, 2.75109, 3.55236, ...","[0.0, 8.33166, 37.94071, 63.53617, 82.18998, 9...","[0.0, 0.07306, 0.36862, 0.64739, 0.79238, 1.04...","[0.11929, 0.39309, 0.65603, 0.91083, 1.17222, ..."
4,0.367255,0.311616,AR9HQ6Y1187FB3C2CB,NaN,None,NaN,Orlando Pops Orchestra,None,Easy Listening: Cartoon Songs,NaN,...,None,b'K\x9b\x81\x00',None,"[""orchestra"", ""musical theater"", ""british"", ""b...","[""ARFMQNW12454A47686"", ""ARKWDUB124207801C0"", ""...","[0.99301, 2.99441, 4.96889, 6.9402, 8.92127, 1...","[0.49266, 0.99301, 1.49336, 1.99371, 2.49406, ...","[0.0, 8.42605, 32.81331, 64.19633, 74.73863, 1...","[0.0, 0.3166, 0.82195, 1.2751, 1.80871, 2.3314...","[0.24248, 0.49266, 0.74283, 0.99301, 1.24318, ..."
